# AIM
## Hierarchical Agglomerative Clustering Implementation

The first step in the algorithm is to create a distance matrix. To find the distances between each of the points, we are using Euclidean distance. Initially, each and every point is a cluster. Then to merge the clusters, we check the entire distance matrix for the minimum distance between two clusters because we are doing HAC with Min link. Once we find the minimum distance between two clusters, we merge these clusters and update the distance matrix by finding out the new distance between the cluster we formed and all the other clusters in the distance matrix. Everytime we merge two clusters, we create a dendogram which shows the link between two merged clusters. We repeat this same process of merging minimum clusters until we get the desired number of cluster which depends on the k-value or until until we merge all objects into a single cluster.

### Importing Dependencies

In [29]:
import numpy as np
import pickle
import time
import sys
from sklearn.decomposition import PCA as sklearnPCA


### Preprocess the input data

**Input Parameters** : file name


**returns**: output matrix X, disease id and the ground truth

In [30]:
def preprocess(filename):
    inpdata = np.genfromtxt(filename,delimiter = '\t')
    X = np.loadtxt(filename,delimiter = '\t', usecols = range(2, inpdata.shape[1]), dtype = 'S15')
    gen_id = np.loadtxt(filename,delimiter = '\t', usecols = 0, dtype = 'S15')
    ground_truth = np.loadtxt(filename,delimiter = '\t', usecols = 1, dtype = 'S15')
    return X, gen_id, ground_truth

## Function to run the Heirarchical Agglomerative Clustering Algorithm

**Input Parameters** : input matrix X, disease id and the number of clusters k.


**returns**: cluster pca matrix and clusters

In [31]:
def agglomerative_clus(gen_id, distance_matrix, k):
    clusters = gen_id.tolist()
    
    cluster_pca = []
    for i in gen_id:
        cluster_pca.append([i])
    while(len(clusters) > k):
#         print (clusters)
        in1, in2 = find_min_indices(distance_matrix)
        left = -1
        right = -1
        if(in1 < in2):
            left = in1
            right = in2
        else:
            right = in1 
            left = in2
        if(type(clusters[left]) == bytes):
            clusters[left] = clusters[left].decode("utf-8") 
        if(type(clusters[right]) == bytes):
            clusters[right] = clusters[right].decode("utf-8") 
        merged_cluster = "("+clusters[left] + "," + clusters[right] + ")"
        clusters[left] = merged_cluster
        cluster_pca[left] = cluster_pca[left] + cluster_pca[right]
        del clusters[right]
        del cluster_pca[right]
        for i in range(0, len(distance_matrix)):
            distance_matrix[left][i] = min(distance_matrix[left][i], distance_matrix[right][i])
            distance_matrix[i][left] = min(distance_matrix[left][i], distance_matrix[right][i])
        distance_matrix = np.delete(distance_matrix, right, 0)
        distance_matrix = np.delete(distance_matrix, right, 1)
    return cluster_pca, clusters

### Draw Scatter Plot using plotly

**Input Parameters** : 2 dimensional data and its label


**prints**: visualized clusters

### Find the indices of cell which has the minimum value for HAC 

**Input Parameters** : distance matrix of size m X n


**returns**: indices i and j

In [32]:
def find_min_indices(distance_matrix):
    index1 = -1
    index2 = -1
    min_elem = sys.maxsize
    for i in range(0, distance_matrix.shape[0]):
        for j in range(0, distance_matrix.shape[1]):
            if(i != j):
                if(distance_matrix[i][j] < min_elem):
                    min_elem = distance_matrix[i][j]
                    index1 = i 
                    index2 = j
    return index1, index2

### Calculate the Jaccard value of predicted data

**Input Parameters** : actual ground truth matrix and predicted ground truth


**returns**: None


**prints**: Jaccard and Rand value on console

In [33]:
def calculateJackard(actual_ground_truth, predicted_ground_truth):
    m00, m01, m10, m11 = 0, 0, 0, 0
    for i in range(0, len(actual_ground_truth)):
        for j in range(0, len(actual_ground_truth)):
            if((actual_ground_truth[i] != actual_ground_truth[j]) and (predicted_ground_truth[i] != predicted_ground_truth[j])):
                m00 += 1
            elif((actual_ground_truth[i] == actual_ground_truth[j]) and (predicted_ground_truth[i] != predicted_ground_truth[j])):
                m01 += 1
            elif((actual_ground_truth[i] != actual_ground_truth[j]) and (predicted_ground_truth[i] == predicted_ground_truth[j])):
                m10 += 1
            elif((actual_ground_truth[i] == actual_ground_truth[j]) and (predicted_ground_truth[i] == predicted_ground_truth[j])):
                m11 += 1
    jaccard = m11 / float(m11 + m10 + m01)
    rand = (m11 + m00) / float(m11 + m10 + m01 + m00)
    print(" Effieciency is : " + str(jaccard*40)),
    print(" Rand is : " + str(rand))

### Run principle component analysis to convert n-dimensional data to 2 dimensions in order to visualize
**Input Parameters** : input matrix X


**returns**: Eigen Vector Y

In [34]:
def runPCA(X):
    sklearn_pca = sklearnPCA(n_components=2)
    Y_sklearn = sklearn_pca.fit_transform(X)
    return Y_sklearn

## Driver program to run the above code

**Input Parameters** : file name and no of clusters


**prints**: Jackard value and scatter plot

In [35]:
def driver(file_name, k_value):
    X, gen_id, ground_truth = preprocess(file_name)
    len_X = X.shape[0]
    #Create the distance matrix for the input data
    distance_matrix = np.zeros((len_X, len_X), dtype='float64')  
    for i in range(0, len_X):
        for j in range(0, len_X):
            if(i != j):
                dis = 0
                for k in range(0, X.shape[1]):
                    dis = dis + np.square(np.subtract(float(X[i][k]), float(X[j][k])))
                distance_matrix[i][j] = np.sqrt(dis) 
                distance_matrix[j][i] = np.sqrt(dis)
    start = time.time()
    clusters_pca, clusters = agglomerative_clus(gen_id, distance_matrix, k_value)
    heirarical_ground_truth = [0]*len(gen_id)
    for i in range(0, len(clusters_pca)):
        for j in clusters_pca[i]:
            heirarical_ground_truth[int(j)-1] = i
    print("Time is : "),
    print("--- %s seconds ---" % (time.time() - start))
    calculateJackard(ground_truth, heirarical_ground_truth)
    Y_pca = runPCA(X)


In [36]:
file_name = "data.txt"
k_value = 1
driver(file_name, k_value)

Time is : 
--- 3.4365322589874268 seconds ---
 Effieciency is : 0.497606898461295
 Rand is : 0.012440172461532376


# Conclusion
We found out that the efficiency comes out to be 0.4976 when we code it via without inbuilt functions of python